In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from collections import Counter

# from sklearn.datasets import load_boston
sns.set_style('darkgrid')

import warnings
warnings.filterwarnings("ignore")

In [2]:
df_elevator = pd.read_csv('/Volumes/Debbie 1T/BDSE35-1_Final Project_2/data/house1004-2.csv') 
df_elevator.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240149 entries, 0 to 240148
Data columns (total 70 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Unnamed: 0     240149 non-null  int64  
 1   Column1        240149 non-null  int64  
 2   鄉鎮市區           240130 non-null  object 
 3   交易標的           240149 non-null  object 
 4   土地位置建物門牌       240149 non-null  object 
 5   土地移轉總面積平方公尺    240149 non-null  float64
 6   都市土地使用分區       218861 non-null  object 
 7   非都市土地使用分區      20391 non-null   object 
 8   非都市土地使用編定      20373 non-null   object 
 9   交易年月日          238012 non-null  object 
 10  交易筆棟數          240149 non-null  object 
 11  移轉層次           239998 non-null  object 
 12  總樓層數           239739 non-null  object 
 13  建物型態           240149 non-null  object 
 14  主要用途           239742 non-null  object 
 15  主要建材           239739 non-null  object 
 16  建築完成年月         212738 non-null  float64
 17  建物移轉總面積平方公尺    240149 non-nul

In [3]:
df = pd.read_csv('/Volumes/Debbie 1T/BDSE35-1_Final Project_2/data/house1004.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240149 entries, 0 to 240148
Data columns (total 68 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Unnamed: 0     240149 non-null  int64  
 1   Column1        240149 non-null  int64  
 2   鄉鎮市區           240130 non-null  object 
 3   交易標的           240149 non-null  object 
 4   土地位置建物門牌       240149 non-null  object 
 5   土地移轉總面積平方公尺    240149 non-null  float64
 6   都市土地使用分區       218861 non-null  object 
 7   非都市土地使用分區      20391 non-null   object 
 8   非都市土地使用編定      20373 non-null   object 
 9   交易年月日          238012 non-null  object 
 10  交易筆棟數          240149 non-null  object 
 11  移轉層次           239998 non-null  object 
 12  總樓層數           239739 non-null  object 
 13  建物型態           240149 non-null  object 
 14  主要用途           239742 non-null  object 
 15  主要建材           239739 non-null  object 
 16  建築完成年月         212738 non-null  float64
 17  建物移轉總面積平方公尺    240149 non-nul

In [4]:
df_2 = pd.read_csv('/Volumes/Debbie 1T/BDSE35-1_Final Project_2/data/kde_results.csv') 
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240133 entries, 0 to 240132
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   編號         240133 non-null  object 
 1   KDE_0.5km  240133 non-null  float64
 2   KDE_1km    240133 non-null  float64
 3   KDE_1.5km  240133 non-null  float64
dtypes: float64(3), object(1)
memory usage: 7.3+ MB


In [5]:
df_2.head()

,編號,KDE_0.5km,KDE_1km,KDE_1.5km
0,RPPPMLRLKHIFFHR07CA,0.010250,0.003468,0.001877
1,RPSPMLQLLHIFFHR37CA,0.010201,0.003466,0.001877
2,RPXNMLNKNHIFFAD97CA,0.010899,0.003453,0.001855
3,RPWNMLKMQHHFFHR47CA,0.010163,0.003474,0.001881
4,RPTOMLNJRHHFFHR37CA,0.010264,0.003471,0.001878


In [6]:
df = df.merge(df_2, on='編號', how='left')

In [7]:
df.head()

,Unnamed: 0,Column1,鄉鎮市區,交易標的,土地位置建物門牌,土地移轉總面積平方公尺,都市土地使用分區,非都市土地使用分區,非都市土地使用編定,交易年月日,...,商,工,農,移轉層次_數字,包含騎樓_編碼,包含屋頂_編碼,包含一二樓_編碼,KDE_0.5km,KDE_1km,KDE_1.5km
0,0,0,中西區,房地(土地+建物),臺南市中西區金華路三段２１８號十五樓之１７,4.11,商,NaN,NaN,2022/9/2 00:00,...,1,0,0,15,0,0,0,0.003601,0.001542,0.000867
1,1,1,北區,房地(土地+建物),臺南市北區南園街１２７巷１５１弄３號,52.00,住,NaN,NaN,2022/8/1 00:00,...,0,0,0,全,0,0,0,0.003506,0.001622,0.000910
2,2,2,佳里區,房地(土地+建物),臺南市佳里區麻興路二段５３１號,314.27,NaN,鄉村區,乙種建築用地,2022/8/4 00:00,...,0,0,0,全,0,0,0,0.000497,0.000467,0.000462
3,3,3,新化區,房地(土地+建物),臺南市新化區中正路６６２巷２２號,61.00,NaN,特定農業區,甲種建築用地,2022/8/1 00:00,...,0,0,0,全,0,0,0,0.000897,0.000846,0.000645
4,4,4,中西區,房地(土地+建物),臺南市中西區南寧街１３６巷９號,62.86,住,NaN,NaN,2022/8/1 00:00,...,0,0,0,全,0,0,0,0.003568,0.001565,0.000875


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240149 entries, 0 to 240148
Data columns (total 71 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Unnamed: 0     240149 non-null  int64  
 1   Column1        240149 non-null  int64  
 2   鄉鎮市區           240130 non-null  object 
 3   交易標的           240149 non-null  object 
 4   土地位置建物門牌       240149 non-null  object 
 5   土地移轉總面積平方公尺    240149 non-null  float64
 6   都市土地使用分區       218861 non-null  object 
 7   非都市土地使用分區      20391 non-null   object 
 8   非都市土地使用編定      20373 non-null   object 
 9   交易年月日          238012 non-null  object 
 10  交易筆棟數          240149 non-null  object 
 11  移轉層次           239998 non-null  object 
 12  總樓層數           239739 non-null  object 
 13  建物型態           240149 non-null  object 
 14  主要用途           239742 non-null  object 
 15  主要建材           239739 non-null  object 
 16  建築完成年月         212738 non-null  float64
 17  建物移轉總面積平方公尺    240149 non-nul

In [9]:
df_columns_cleaned = df.drop(columns=['Unnamed: 0', 'Column1', '交易標的', '土地移轉總面積平方公尺', '都市土地使用分區', '非都市土地使用分區', '非都市土地使用編定', '交易筆棟數', '移轉層次', '總樓層數', '建物型態', '主要用途', '主要建材', '建物現況格局-房', '建物現況格局-廳', '建物現況格局-衛', '建物現況格局-隔間', '車位類別', '車位移轉總面積(平方公尺)', '車位總價元', '主建物面積', '附屬建物面積', '陽台面積', '移轉編號', '車位移轉總面積平方公尺', '建案名稱', '棟及號', '解約情形','移轉層次_數字'])
df_columns_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240149 entries, 0 to 240148
Data columns (total 42 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   鄉鎮市區         240130 non-null  object 
 1   土地位置建物門牌     240149 non-null  object 
 2   交易年月日        238012 non-null  object 
 3   建築完成年月       212738 non-null  float64
 4   建物移轉總面積平方公尺  240149 non-null  float64
 5   有無管理組織       240149 non-null  object 
 6   總價元          240149 non-null  int64  
 7   單價元平方公尺      238544 non-null  float64
 8   備註           65433 non-null   object 
 9   編號           240149 non-null  object 
 10  電梯           74531 non-null   object 
 11  緯度           240149 non-null  float64
 12  經度           240149 non-null  float64
 13  bad_count    238012 non-null  float64
 14  good_count   238012 non-null  float64
 15  建築型態清整       240149 non-null  object 
 16  陽台有無         240149 non-null  int64  
 17  是否含額外土地      240149 non-null  int64  
 18  建築型態_住商大樓    240149 non-

In [10]:
df_columns_cleaned = pd.concat([df_columns_cleaned,df_elevator['電梯_數值化']], axis=1)
df_columns_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240149 entries, 0 to 240148
Data columns (total 43 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   鄉鎮市區         240130 non-null  object 
 1   土地位置建物門牌     240149 non-null  object 
 2   交易年月日        238012 non-null  object 
 3   建築完成年月       212738 non-null  float64
 4   建物移轉總面積平方公尺  240149 non-null  float64
 5   有無管理組織       240149 non-null  object 
 6   總價元          240149 non-null  int64  
 7   單價元平方公尺      238544 non-null  float64
 8   備註           65433 non-null   object 
 9   編號           240149 non-null  object 
 10  電梯           74531 non-null   object 
 11  緯度           240149 non-null  float64
 12  經度           240149 non-null  float64
 13  bad_count    238012 non-null  float64
 14  good_count   238012 non-null  float64
 15  建築型態清整       240149 non-null  object 
 16  陽台有無         240149 non-null  int64  
 17  是否含額外土地      240149 non-null  int64  
 18  建築型態_住商大樓    240149 non-

In [11]:
# 使用 rename 方法重命名欄位
df_columns_cleaned = df_columns_cleaned.rename(columns={
    '包含騎樓_編碼': '移轉層次_騎樓',
    '包含屋頂_編碼': '移轉層次_屋頂',
    '包含一二樓_編碼': '移轉層次_一二樓',
    '電梯_數值化':'有無電梯'
})
df_columns_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240149 entries, 0 to 240148
Data columns (total 43 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   鄉鎮市區         240130 non-null  object 
 1   土地位置建物門牌     240149 non-null  object 
 2   交易年月日        238012 non-null  object 
 3   建築完成年月       212738 non-null  float64
 4   建物移轉總面積平方公尺  240149 non-null  float64
 5   有無管理組織       240149 non-null  object 
 6   總價元          240149 non-null  int64  
 7   單價元平方公尺      238544 non-null  float64
 8   備註           65433 non-null   object 
 9   編號           240149 non-null  object 
 10  電梯           74531 non-null   object 
 11  緯度           240149 non-null  float64
 12  經度           240149 non-null  float64
 13  bad_count    238012 non-null  float64
 14  good_count   238012 non-null  float64
 15  建築型態清整       240149 non-null  object 
 16  陽台有無         240149 non-null  int64  
 17  是否含額外土地      240149 non-null  int64  
 18  建築型態_住商大樓    240149 non-

In [12]:
df_3= pd.read_csv('/Volumes/Debbie 1T/BDSE35-1_Final Project_2/data/house1016_withKDE_Regioin.csv')
df_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237834 entries, 0 to 237833
Data columns (total 39 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   鄉鎮市區         237834 non-null  object 
 1   土地位置建物門牌     237834 non-null  object 
 2   交易年月日        237834 non-null  object 
 3   建物移轉總面積平方公尺  237834 non-null  float64
 4   有無管理組織       237834 non-null  object 
 5   總價元          237834 non-null  int64  
 6   單價元平方公尺      236230 non-null  float64
 7   編號           237834 non-null  object 
 8   電梯           74518 non-null   object 
 9   緯度           237834 non-null  float64
 10  經度           237834 non-null  float64
 11  bad_count    237834 non-null  float64
 12  good_count   237834 non-null  float64
 13  建築型態清整       237834 non-null  object 
 14  陽台有無         237834 non-null  int64  
 15  是否含額外土地      237834 non-null  int64  
 16  建築型態_住商大樓    237834 non-null  int64  
 17  建築型態_公寓      237834 non-null  int64  
 18  建築型態_其他      237834 non-

In [13]:
# 提取需要的列
df_region = df_3[['編號', 'Region']]

# 合并 'Region' 列到 df
df_columns_cleaned = df_columns_cleaned.merge(df_region, on='編號', how='left')

df_columns_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240149 entries, 0 to 240148
Data columns (total 44 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   鄉鎮市區         240130 non-null  object 
 1   土地位置建物門牌     240149 non-null  object 
 2   交易年月日        238012 non-null  object 
 3   建築完成年月       212738 non-null  float64
 4   建物移轉總面積平方公尺  240149 non-null  float64
 5   有無管理組織       240149 non-null  object 
 6   總價元          240149 non-null  int64  
 7   單價元平方公尺      238544 non-null  float64
 8   備註           65433 non-null   object 
 9   編號           240149 non-null  object 
 10  電梯           74531 non-null   object 
 11  緯度           240149 non-null  float64
 12  經度           240149 non-null  float64
 13  bad_count    238012 non-null  float64
 14  good_count   238012 non-null  float64
 15  建築型態清整       240149 non-null  object 
 16  陽台有無         240149 non-null  int64  
 17  是否含額外土地      240149 non-null  int64  
 18  建築型態_住商大樓    240149 non-

In [14]:
df_columns_cleaned.isna().sum()

鄉鎮市區               19
土地位置建物門牌            0
交易年月日            2137
建築完成年月          27411
建物移轉總面積平方公尺         0
有無管理組織              0
總價元                 0
單價元平方公尺          1605
備註             174716
編號                  0
電梯             165618
緯度                  0
經度                  0
bad_count        2137
good_count       2137
建築型態清整              0
陽台有無                0
是否含額外土地             0
建築型態_住商大樓           0
建築型態_公寓             0
建築型態_其他             0
建築型態_透天厝            0
房地總價元               0
格局比例_房           4769
格局比例_廳           4769
格局比例_衛           4769
格局比例_隔間          4769
是否包含車位              0
建材_磚石               0
建材_鋼筋               0
建材_鋼骨               0
建材_竹木               0
住                   0
商                   0
工                   0
農                   0
移轉層次_騎樓             0
移轉層次_屋頂             0
移轉層次_一二樓            0
KDE_0.5km          16
KDE_1km            16
KDE_1.5km          16
有無電梯                0
Region          14447
dtype: int64

In [15]:
# 篩選出 '鄉鎮市區' 為 NaN 的 '土地位置建物門牌' 欄位
nan_town_area = df_columns_cleaned[df_columns_cleaned['鄉鎮市區'].isna()][['鄉鎮市區','土地位置建物門牌']]

# 顯示結果
print(nan_town_area)

       鄉鎮市區               土地位置建物門牌
204016  NaN  臺南市官田區隆本里和平街二段26巷1弄3號
218820  NaN               臺南市東區平實路
218822  NaN               臺南市東區平實路
218824  NaN               臺南市東區平實路
218828  NaN               臺南市東區平實路
218829  NaN               臺南市東區平實路
218832  NaN               臺南市東區平實路
218974  NaN               臺南市東區平實路
219117  NaN               臺南市東區平實路
219118  NaN               臺南市東區平實路
219119  NaN               臺南市東區平實路
219120  NaN               臺南市東區平實路
219121  NaN               臺南市東區平實路
219122  NaN               臺南市東區平實路
219123  NaN               臺南市東區平實路
219124  NaN               臺南市東區平實路
219125  NaN               臺南市東區平實路
219126  NaN               臺南市東區平實路
219127  NaN               臺南市東區平實路


In [17]:
import re

# 定義一個函數來從 '土地位置建物門牌' 中提取 '市' 後面的區域
def extract_district(address):
    match = re.search(r'市(.+?區)', address)
    if match:
        return match.group(1)
    return None

# 將 '鄉鎮市區' 欄位為 NaN 的行補充 '土地位置建物門牌' 中的區域名稱
df_columns_cleaned.loc[df_columns_cleaned['鄉鎮市區'].isna(), '鄉鎮市區'] = df_columns_cleaned['土地位置建物門牌'].apply(extract_district)

# 檢查結果
print(df_columns_cleaned[['鄉鎮市區', '土地位置建物門牌']].head())

  鄉鎮市區               土地位置建物門牌
0  中西區  臺南市中西區金華路三段２１８號十五樓之１７
1   北區     臺南市北區南園街１２７巷１５１弄３號
2  佳里區        臺南市佳里區麻興路二段５３１號
3  新化區       臺南市新化區中正路６６２巷２２號
4  中西區        臺南市中西區南寧街１３６巷９號


In [18]:
df_columns_cleaned.isna().sum()

鄉鎮市區                0
土地位置建物門牌            0
交易年月日            2137
建築完成年月          27411
建物移轉總面積平方公尺         0
有無管理組織              0
總價元                 0
單價元平方公尺          1605
備註             174716
編號                  0
電梯             165618
緯度                  0
經度                  0
bad_count        2137
good_count       2137
建築型態清整              0
陽台有無                0
是否含額外土地             0
建築型態_住商大樓           0
建築型態_公寓             0
建築型態_其他             0
建築型態_透天厝            0
房地總價元               0
格局比例_房           4769
格局比例_廳           4769
格局比例_衛           4769
格局比例_隔間          4769
是否包含車位              0
建材_磚石               0
建材_鋼筋               0
建材_鋼骨               0
建材_竹木               0
住                   0
商                   0
工                   0
農                   0
移轉層次_騎樓             0
移轉層次_屋頂             0
移轉層次_一二樓            0
KDE_0.5km          16
KDE_1km            16
KDE_1.5km          16
有無電梯                0
Region          14447
dtype: int64

In [21]:
# nan_deal_date = df_columns_cleaned[df_columns_cleaned['交易年月日'].isna()][['交易年月日','建築完成年月']]


# 篩選出 '交易年月日' 為 NaN 的行
nan_deal_date = df_columns_cleaned[df_columns_cleaned['交易年月日'].isna()][['交易年月日', '建築完成年月']]

# 從 df_columns_cleaned 中移除這些行
df_columns_cleaned = df_columns_cleaned.drop(nan_deal_date.index)


In [23]:
# 找出 [單價元平方公尺] 為 NaN 或者為 0 的行
mask = (df_columns_cleaned['單價元平方公尺'].isna()) | (df_columns_cleaned['單價元平方公尺'] == 0)

# 計算 [房地總價元] / [建物移轉總面積平方公尺]，針對滿足條件的行進行計算
df_columns_cleaned.loc[mask, '單價元平方公尺'] = df_columns_cleaned.loc[mask, '房地總價元'] / df_columns_cleaned.loc[mask, '建物移轉總面積平方公尺']

# 查看更新後的數據
print(df_columns_cleaned[mask][['房地總價元', '建物移轉總面積平方公尺', '單價元平方公尺']].head())

         房地總價元  建物移轉總面積平方公尺       單價元平方公尺
1133  38130000       629.70  60552.644116
1140  11700000       197.28  59306.569343
1305   9511976       334.47  28438.951176
1791   1800000        82.52  21812.893844
1937  41580000       865.86  48021.620123


In [24]:
df_columns_cleaned.isna().sum()

鄉鎮市區                0
土地位置建物門牌            0
交易年月日               0
建築完成年月          25275
建物移轉總面積平方公尺         0
有無管理組織              0
總價元                 0
單價元平方公尺             0
備註             172613
編號                  0
電梯             163482
緯度                  0
經度                  0
bad_count           0
good_count          0
建築型態清整              0
陽台有無                0
是否含額外土地             0
建築型態_住商大樓           0
建築型態_公寓             0
建築型態_其他             0
建築型態_透天厝            0
房地總價元               0
格局比例_房           4757
格局比例_廳           4757
格局比例_衛           4757
格局比例_隔間          4757
是否包含車位              0
建材_磚石               0
建材_鋼筋               0
建材_鋼骨               0
建材_竹木               0
住                   0
商                   0
工                   0
農                   0
移轉層次_騎樓             0
移轉層次_屋頂             0
移轉層次_一二樓            0
KDE_0.5km          16
KDE_1km            16
KDE_1.5km          16
有無電梯                0
Region          12310
dtype: int64

In [25]:
# 換算坪數
conversion_factor = 3.305785

# 將「建物移轉總面積平方公尺」轉換成坪
df_columns_cleaned['建物移轉總面積坪'] = df_columns_cleaned['建物移轉總面積平方公尺'] * conversion_factor
# 將「單價元平方公尺」轉換成「單價元每坪」
df_columns_cleaned['單價元每坪'] = df_columns_cleaned['單價元平方公尺'] * conversion_factor
df_columns_cleaned['房地總價元(坪)'] = df_columns_cleaned['建物移轉總面積坪'] * df_columns_cleaned['單價元每坪']

In [26]:
df_columns_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238012 entries, 0 to 238011
Data columns (total 47 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   鄉鎮市區         238012 non-null  object 
 1   土地位置建物門牌     238012 non-null  object 
 2   交易年月日        238012 non-null  object 
 3   建築完成年月       212737 non-null  float64
 4   建物移轉總面積平方公尺  238012 non-null  float64
 5   有無管理組織       238012 non-null  object 
 6   總價元          238012 non-null  int64  
 7   單價元平方公尺      238012 non-null  float64
 8   備註           65399 non-null   object 
 9   編號           238012 non-null  object 
 10  電梯           74530 non-null   object 
 11  緯度           238012 non-null  float64
 12  經度           238012 non-null  float64
 13  bad_count    238012 non-null  float64
 14  good_count   238012 non-null  float64
 15  建築型態清整       238012 non-null  object 
 16  陽台有無         238012 non-null  int64  
 17  是否含額外土地      238012 non-null  int64  
 18  建築型態_住商大樓    238012 non-

In [27]:
df_columns_cleaned.isna().sum()

鄉鎮市區                0
土地位置建物門牌            0
交易年月日               0
建築完成年月          25275
建物移轉總面積平方公尺         0
有無管理組織              0
總價元                 0
單價元平方公尺             0
備註             172613
編號                  0
電梯             163482
緯度                  0
經度                  0
bad_count           0
good_count          0
建築型態清整              0
陽台有無                0
是否含額外土地             0
建築型態_住商大樓           0
建築型態_公寓             0
建築型態_其他             0
建築型態_透天厝            0
房地總價元               0
格局比例_房           4757
格局比例_廳           4757
格局比例_衛           4757
格局比例_隔間          4757
是否包含車位              0
建材_磚石               0
建材_鋼筋               0
建材_鋼骨               0
建材_竹木               0
住                   0
商                   0
工                   0
農                   0
移轉層次_騎樓             0
移轉層次_屋頂             0
移轉層次_一二樓            0
KDE_0.5km          16
KDE_1km            16
KDE_1.5km          16
有無電梯                0
Region          12310
建物移轉總面積坪            0
單價元每坪     

In [28]:
df_columns_cleaned.to_csv('/Volumes/Debbie 1T/BDSE35-1_Final Project_2/data/實價登錄房價_238012row_cleaned.csv',index=False)